### A zero dimentional global energy balance model including ice albedo feedback

In [67]:
# Import relevant packages

import numpy as np
from scipy.integrate import solve_ivp
from bokeh.io import show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from bokeh.palettes import Colorblind
output_notebook()

Loading BokehJS ...

In [91]:
# Define parameters

S0 = 1366.0 / 4 # W/m^2
alpha = 2.0
beta = 5.0
T_init = 2.0
emissivity = 0.6
sigma = 5.67e-8
timesteps = 500
timepoints = np.linspace(0, 10, timesteps)
tempsteps = 350
temperature = np.linspace(-150, 200, tempsteps)
albedo = np.zeros(tempsteps)
rad_incoming = np.zeros(tempsteps)
rad_outgoing = np.zeros(tempsteps)

In [92]:
# Model definitions

# Define energy balance model
def EnergyBalanceEqn(t, y):
    T = y
    rin = IncomingRad(T, 1)
    rout = OutgoingRad(T)
    dTdt = [rin - rout]
    return dTdt

# Define Albedo
def CalcAlbedo(T):
    a_max = 0.9
    a_min = 0.1
    T0 = 255 - 273
    A = (a_max - a_min) / 2
    B = (a_max + a_min) / 2
    return A * np.tanh(alpha * (T / T0)) + B

# Define incoming radiation
def IncomingRad(T, noiseFlag):
    if noiseFlag == 1:
        return S0 * (1.0 - CalcAlbedo(T) + beta * np.random.randn())
    else:
        return S0 * (1.0 - CalcAlbedo(T))

# Define outgoing radiation
def OutgoingRad(T):
    return emissivity * sigma * (T + 273)**4

In [93]:
# Solve the energy balance model 

timelist = []
templist = []
for i in range(4):
    sol = solve_ivp(EnergyBalanceEqn, (0, 10), [T_init], method='RK45', t_eval=timepoints)
    timelist.append(sol.t)
    templist.append(sol.y[0, :])

In [94]:
# Calculate albedo and radiation for given tempeature range (no stochastic noise)

albedo = CalcAlbedo(temperature)
rad_incoming = IncomingRad(temperature, 0)
rad_outgoing = OutgoingRad(temperature)
rad_diff = rad_incoming - rad_outgoing

In [95]:
# Plot results

# Figure 1
p1 = figure(plot_width=900, plot_height=400, x_axis_label='Time', y_axis_label='Temperature',
           title='Zero-dimensional energy balance model')
p1.multi_line(timelist, templist, color=Colorblind[4][:])
p1.toolbar.logo = None
p1.toolbar_location = None

# Figure 2
p2 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Albedo',
           title='S0 = ' + str(S0) + ' W/m^2')
p2.line(temperature, albedo, color=Colorblind[3][0])
p2.toolbar.logo = None
p2.toolbar_location = None

p3 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Radiation')
p3.line(temperature, rad_incoming, color=Colorblind[6][5], legend='incoming')
p3.line(temperature, rad_outgoing, color=Colorblind[6][0], legend='outgoing')
p3.legend.location = 'top_left'  
p3.toolbar.logo = None
p3.toolbar_location = None

p4 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Radiation',
           title='dT/dt = incoming-outgoing')
p4.line(temperature, rad_diff, color=Colorblind[8][5])
p4.line(temperature, np.zeros(tempsteps), color=Colorblind[8][7])
p4.toolbar.logo = None
p4.toolbar_location = None

# Display plots
show(column(p1, row(p2, p3, p4)))